In [ ]:
!pip install instructor
!pip install langchain-groq
!pip install langchain
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.6/345.6 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: jiter
    Found existing installation: jiter 0.9.0
    Uninstalling jiter-0.9.0:
      Successfully uninstalled jiter-0.9.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00


In [70]:
import instructor
from langchain_groq import ChatGroq
from typing import Iterable, List, Optional
from enum import Enum
from pydantic import BaseModel

In [71]:
llm = ChatGroq(
    api_key="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",
    model_name="deepseek-r1-distill-llama-70b"
  )

In [72]:
prompt=  """
Alice: Hey team, we have several critical tasks we need to tackle for the upcoming release. First, we need to work on improving the authentication system. It's a top priority.
Bob: Got it, Alice. I can take the lead on the authentication improvements. Are there any specific areas you want me to focus on?
Alice: Good question, Bob. We need both a front-end revamp and back-end optimization. So basically, two sub-tasks.
Carol: I can help with the front-end part of the authentication system.
Bob: Great, Carol. I'll handle the back-end optimization then.
Alice: Perfect. Now, after the authentication system is improved, we have to integrate it with our new billing system as a sperate task. That's a medium priority task.
Carol: Is the new billing system already in place?
Alice: No, it's actually another task. So it's a dependency for the integration task. Bob, can you also handle the billing system?
Bob: Sure, but I'll need to complete the back-end optimization of the authentication system first, so it's dependent on that.
Alice: Understood. Lastly, we also need to update our user documentation to reflect all these changes. It's a low-priority task but still important.
Carol: I can take that on once the front-end changes for the authentication system are done. So, it would be dependent on that.
Alice: Sounds like a plan. Let's get these tasks modeled out and get started."""

# **Summarization**

In [73]:
from langchain.schema import HumanMessage
from langchain.prompts import PromptTemplate
prompt_template = PromptTemplate(
    input_variables=["transcript"],
    template="""
You are an intelligent assistant designed to summarize business meetings.
Below is the transcript of a meeting. Your task is to generate a clear, concise, and comprehensive summary that includes all important details.

Focus on the following only:
- Key discussion points and topics covered
- Important decisions made
Avoid repeating dialogue and keep the summary professional and well-structured.

Meeting Transcript:
{transcript}

Generate the summary below:
"""
)


def summarize_meeting(transcript_text: str) -> str:
    prompt = prompt_template.format(transcript=transcript_text)
    response = llm([HumanMessage(content=prompt)])
    return response.content

summary=summarize_meeting(prompt)

In [76]:
import re
summary=re.sub(r"<think>.*?</think>", "", summary, flags=re.DOTALL).strip()
summary = summary.replace("*", "")
print(summary)

Meeting Summary

- Key Discussion Points and Topics Covered:
  1. Authentication System Improvement:  
     - Identified as the top priority task.  
     - Sub-tasks include front-end revamp and back-end optimization.  
     - Bob will lead the back-end optimization, while Carol will handle the front-end part.  

  2. Integration with New Billing System:  
     - This is a medium-priority task.  
     - The integration is dependent on the completion of the authentication system improvements.  
     - Bob will also handle the development of the new billing system, with the work starting after the back-end optimization of the authentication system is completed.  

  3. User Documentation Update:  
     - Identified as a low-priority task but still important.  
     - Carol will take charge of updating the user documentation once the front-end changes for the authentication system are completed.  

- Important Decisions Made:
  - Tasks and responsibilities were assigned as follows:  
    

#**Action Item Classification**

In [77]:
#Model
class PriorityEnum(str, Enum):
    urgent="1"
    high = "2"
    medium = "3"
    low = "4"

class Subtask(BaseModel):
    """Correctly resolved subtask from the given transcript"""
    id: int
    name: str
    description: str
    priority: PriorityEnum
    assignees: List[str]


class Ticket(BaseModel):
    """Correctly resolved ticket from the given transcript"""
    id: int
    name: str
    description: str
    priority: PriorityEnum
    assignees: List[str]
    subtasks: Optional[List[Subtask]]
    dependencies: Optional[List[int]]

class TicketList(BaseModel):
    tickets: List[Ticket]


In [78]:
client = llm.with_structured_output(TicketList)
def generate(data)-> Iterable[Ticket]:
  messages = [
    ("system", "The following is a transcript of a meeting..."),
    ("human", f"Create the action items for the following transcript: {data}")
    ]
  output = client.invoke(messages)
  return output.tickets


In [79]:
prediction = generate(prompt)
print(prediction)

[Ticket(id=1, name='Improve Authentication System', description='Critical task to improve the authentication system', priority=<PriorityEnum.urgent: '1'>, assignees=['Bob'], subtasks=[Subtask(id=1, name='Front-end Revamp', description='Front-end improvements for authentication system', priority=<PriorityEnum.urgent: '1'>, assignees=['Carol']), Subtask(id=2, name='Back-end Optimization', description='Back-end optimization for authentication system', priority=<PriorityEnum.urgent: '1'>, assignees=['Bob'])], dependencies=None), Ticket(id=2, name='Integrate with Billing System', description='Integrate authentication system with new billing system', priority=<PriorityEnum.high: '2'>, assignees=['Bob'], subtasks=None, dependencies=[2]), Ticket(id=3, name='Update User Documentation', description='Update user documentation to reflect changes', priority=<PriorityEnum.low: '4'>, assignees=['Carol'], subtasks=None, dependencies=[1])]


In [83]:
for ticket in prediction:
        print(f"\nTicket {ticket.id}: {ticket.name}")
        print(f"   Description: {ticket.description}")
        print(f"   Priority: {ticket.priority.name.capitalize()}")
        print(f"   Assignees: {', '.join(ticket.assignees)}")

        if ticket.dependencies:
            print(f"   Dependencies: {ticket.dependencies}")

        if ticket.subtasks:
            print("   Subtasks:")
            for sub in ticket.subtasks:
                print(f"     - [{sub.id}] {sub.name}")
                print(f"       Description: {sub.description}")
                print(f"       Priority: {sub.priority.name.capitalize()}")
                print(f"       Assignees: {', '.join(sub.assignees)}")



Ticket 1: Improve Authentication System
   Description: Critical task to improve the authentication system
   Priority: Urgent
   Assignees: Bob
   Subtasks:
     - [1] Front-end Revamp
       Description: Front-end improvements for authentication system
       Priority: Urgent
       Assignees: Carol
     - [2] Back-end Optimization
       Description: Back-end optimization for authentication system
       Priority: Urgent
       Assignees: Bob

Ticket 2: Integrate with Billing System
   Description: Integrate authentication system with new billing system
   Priority: High
   Assignees: Bob
   Dependencies: [2]

Ticket 3: Update User Documentation
   Description: Update user documentation to reflect changes
   Priority: Low
   Assignees: Carol
   Dependencies: [1]


#**Clickup Integration**

In [84]:
from langchain_community.agent_toolkits.clickup.toolkit import ClickupToolkit
from langchain_community.utilities.clickup import ClickupAPIWrapper

In [ ]:
from google.colab import userdata

oauth_client_id = "LI26TABAZH64X2UMU32Q3MOHC84TRJD9"
oauth_client_secret = userdata.get('FMVO016Q5R4JCG78SA299FTT8DLR5NWHFFV37PJ4L58MO6FCZRW3L02MUF0852PK')
redirect_uri = "https://google.com"

print("Click this link, select your workspace, click `Connect Workspace`")
print(ClickupAPIWrapper.get_access_code_url(oauth_client_id, redirect_uri))

Click this link, select your workspace, click `Connect Workspace`
https://app.clickup.com/api?client_id=LI26TABAZH64X2UMU32Q3MOHC84TRJD9&redirect_uri=https://google.com


In [ ]:
code = userdata.get('BR813KIXRWXK863I72UGUDU332BY36PT')

In [ ]:
access_token = ClickupAPIWrapper.get_access_token(
    oauth_client_id, oauth_client_secret, code
)

In [ ]:
clickup_api_wrapper = ClickupAPIWrapper(access_token=access_token)
toolkit = ClickupToolkit.from_clickup_api_wrapper(clickup_api_wrapper)
print(
    f"Found team_id: {clickup_api_wrapper.team_id}.")

Found team_id: 9016845074.


In [ ]:
import requests

url = "https://api.clickup.com/api/v2/list/901607089572/task?custom_task_ids=true&team_id=9016845074"

headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "Authorization": "55398888_6a502008f0daf3f52e28aad2f5b686ff800b90cddc9aec60f9a38a5febb744fb"
}

In [ ]:
for ticket in prediction:
    parent_task = {
        "name": ticket.name,
        "description": ticket.description,
        "tags": ticket.assignees,
        "status": "to do",
        "priority": ticket.priority.value,



    }
    response = requests.post(url, json=parent_task, headers=headers)
    print(response.text)

    if ticket.subtasks:
        parent_id = response.json()['id']
        for subtask in ticket.subtasks:
            subtask_task = {
                "name": subtask.name,
                "description": subtask.description,
                "tags": subtask.assignees,
                "status": "to do",
                "priority": subtask.priority.value,
                "parent":parent_id,
            }
            response = requests.post(url, json=subtask_task, headers=headers)
            print(response.text)

{"id":"86cybkqcw","custom_id":null,"custom_item_id":0,"name":"Improve Authentication System","text_content":"Top priority task to improve the authentication system with front-end and back-end improvements.","description":"Top priority task to improve the authentication system with front-end and back-end improvements.","status":{"id":"p90163169231_DfzAMRIW","status":"to do","color":"#d3d3d3","orderindex":0,"type":"open"},"orderindex":"54.00000000000000000000000000000000","date_created":"1742552252339","date_updated":"1742552252339","date_closed":null,"date_done":null,"archived":false,"creator":{"id":55398888,"username":"Shalavya Agrawal","color":"","email":"shalavyaagrawal@gmail.com","profilePicture":null},"assignees":[],"group_assignees":[],"watchers":[{"id":55398888,"username":"Shalavya Agrawal","color":"","initials":"SA","email":"shalavyaagrawal@gmail.com","profilePicture":null}],"checklists":[],"tags":[{"name":"bob","tag_fg":"#aa2fff","tag_bg":"#aa2fff","creator":55398888}],"parent"